In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install ta
#데이터셋을 만들어 주는 함수
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM,Dropout,GRU
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

import pandas as pd
df_result = pd.DataFrame(columns = ['유닛수', 'DropOut', '층 개수', '최적 개수', '최소 rmse', '최적 예측값'] )
idx = 0
dropOut = [0, 0.2, 0.3]
unitNum = [16, 64, 128]
layerNum = [1, 2]

for drop in dropOut:
  for unit in unitNum:
    for layer in layerNum:
      df = pd.read_csv("/content/drive/MyDrive/HEM/data/_건강관리_삼성바이오로직스.csv")
      df_columns = ['Date','Open', 'High', 'Low', 'Close', 'Volume']
      df.columns = df_columns
      df = df.sort_values('Date')
      df['Date']= df['Date'].astype('str')
      from datetime import datetime
      df['Date'] = pd.to_datetime(df['Date'])
      df.set_index('Date',inplace=True)

      ma = [5,20,60,120]
      for days in ma:
          df['ma_'+str(days)] = df['Close'].rolling(window = days).mean()
          df['vma_'+str(days)] = df['Volume'].rolling(window = days).mean()

      df.dropna(inplace=True)


      import ta

      H, L, C, V = df['High'], df['Low'], df['Close'], df['Volume']

      # df['bol_high'] = ta.volatility.bollinger_hband(C)
      # df['bol_low']  = ta.volatility.bollinger_lband(C)
      df['MFI'] = ta.volume.money_flow_index(
          high=H, low=L, close=C, volume=V, fillna=True)

      df['ADI'] = ta.volume.acc_dist_index(
          high=H, low=L, close=C, volume=V, fillna=True)

      df['OBV'] = ta.volume.on_balance_volume(close=C, volume=V, fillna=True)
      df['CMF'] = ta.volume.chaikin_money_flow(
          high=H, low=L, close=C, volume=V, fillna=True)

      df['FI'] = ta.volume.force_index(close=C, volume=V, fillna=True)
      df['EOM, EMV'] = ta.volume.ease_of_movement(
          high=H, low=L, volume=V, fillna=True)

      df['VPT'] = ta.volume.volume_price_trend(close=C, volume=V, fillna=True)
      df['NVI'] = ta.volume.negative_volume_index(close=C, volume=V, fillna=True)
      df['VMAP'] = ta.volume.volume_weighted_average_price(
          high=H, low=L, close=C, volume=V, fillna=True)

      # Volatility
      df['ATR'] = ta.volatility.average_true_range(
          high=H, low=L, close=C, fillna=True)
      df['BHB'] = ta.volatility.bollinger_hband(close=C, fillna=True)
      df['BLB'] = ta.volatility.bollinger_lband(close=C, fillna=True)
      df['KCH'] = ta.volatility.keltner_channel_hband(
          high=H, low=L, close=C, fillna=True)
      df['KCL'] = ta.volatility.keltner_channel_lband(
          high=H, low=L, close=C, fillna=True)
      df['KCM'] = ta.volatility.keltner_channel_mband(
          high=H, low=L, close=C, fillna=True)
      df['DCH'] = ta.volatility.donchian_channel_hband(
          high=H, low=L, close=C, fillna=True)
      df['DCL'] = ta.volatility.donchian_channel_lband(
          high=H, low=L, close=C, fillna=True)
      df['DCM'] = ta.volatility.donchian_channel_mband(
          high=H, low=L, close=C, fillna=True)
      df['UI'] = ta.volatility.ulcer_index(close=C, fillna=True)
      # Trend
      df['SMA'] = ta.trend.sma_indicator(close=C, fillna=True)
      df['EMA'] = ta.trend.ema_indicator(close=C, fillna=True)
      df['WMA'] = ta.trend.wma_indicator(close=C, fillna=True)
      df['MACD'] = ta.trend.macd(close=C, fillna=True)
      df['ADX'] = ta.trend.adx(high=H, low=L, close=C, fillna=True)
      df['-VI'] = ta.trend.vortex_indicator_neg(
          high=H, low=L, close=C, fillna=True)
      df['+VI'] = ta.trend.vortex_indicator_pos(
          high=H, low=L, close=C, fillna=True)
      df['TRIX'] = ta.trend.trix(close=C, fillna=True)
      df['MI'] = ta.trend.mass_index(high=H, low=L, fillna=True)
      df['CCI'] = ta.trend.cci(high=H, low=L, close=C, fillna=True)
      df['DPO'] = ta.trend.dpo(close=C, fillna=True)
      df['KST'] = ta.trend.kst(close=C, fillna=True)
      df['Ichimoku'] = ta.trend.ichimoku_a(high=H, low=L, fillna=True)
      df['Parabolic SAR'] = ta.trend.psar_down(
          high=H, low=L, close=C, fillna=True)
      df['STC'] = ta.trend.stc(close=C, fillna=True)
      # Momentum
      df['RSI'] = ta.momentum.rsi(close=C, fillna=True)
      df['SRSI'] = ta.momentum.stochrsi(close=C, fillna=True)
      df['TSI'] = ta.momentum.tsi(close=C, fillna=True)
      df['UO'] = ta.momentum.ultimate_oscillator(
          high=H, low=L, close=C, fillna=True)
      df['SR'] = ta.momentum.stoch(close=C, high=H, low=L, fillna=True)
      df['WR'] = ta.momentum.williams_r(high=H, low=L, close=C, fillna=True)
      df['AO'] = ta.momentum.awesome_oscillator(high=H, low=L, fillna=True)
      df['KAMA'] = ta.momentum.kama(close=C, fillna=True)
      df['ROC'] = ta.momentum.roc(close=C, fillna=True)
      df['PPO'] = ta.momentum.ppo(close=C, fillna=True)
      df['PVO'] = ta.momentum.pvo(volume=V, fillna=True)


      from sklearn.preprocessing import MinMaxScaler

      # 피처값 스케일링
      scaler = MinMaxScaler()

      scaled_df = scaler.fit_transform(df.drop(['Close'], axis=1))
      scaled_df = pd.DataFrame(scaled_df, columns = df.drop(['Close'], axis=1).columns)

      scaler1 = MinMaxScaler()

      temp = scaler1.fit_transform(df['Close'].values.reshape(-1,1))

      scaled_df.insert(3, 'Close',temp)
      scaled_df

      df = scaled_df



      import numpy as np

      def make_dataset(feature, label, window_size = 20):
        
        feature_list = []
        label_list = []
        
        for i in range(len(feature) - window_size) :
          feature_list.append(feature.iloc[i:i+window_size])
          label_list.append(label.iloc[i+window_size])
        
        return np.array(feature_list), np.array(label_list)

      WINDOW_SIZE=20

      result = []
      predict = []


      ####최근 20개의 데이터만
      pred_feature = df[-WINDOW_SIZE :]
      pred_feature = pred_feature.to_numpy()

      pred_feature = pred_feature.reshape((1,20,-1))
      # pred_feature.shape

      import random as rn
      import tensorflow as tf


      for i in range(1, len(df)//1000+2):

        # 데이터 불러오기
        df_copy = df
    
        # 원하는 크기로 데이터 자르기
        df_copy = df_copy[-i*1000:]
        # 피처값, 타켓 스케일링
        
        TEST_SIZE = int(len(df_copy) * 0.7)
        
        train = df_copy[:TEST_SIZE]
        test = df_copy[TEST_SIZE:]

##200일의 데이터를 얼마나 잘 예측하느냐
        
        # feature_cols = df.drop('Close', axis =1).columns
        feature_cols = df.columns
        label_cols = ['Close']

        train_feature = train[feature_cols]
        train_label = train[label_cols]

        train_feature, train_label = make_dataset(train_feature, train_label,20)

        from sklearn.model_selection import train_test_split
        x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2, shuffle = False)
        
        
        test_feature = test[feature_cols]
        test_label = test[label_cols]
        test_feature , test_label = make_dataset(test_feature, test_label,20)
        
        
        from keras.layers.normalization import batch_normalization

        model = Sequential()
        if layer == 1:
            model.add(GRU(unit, dropout = drop, recurrent_dropout = drop,
                      input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                      return_sequences = False, 
                      activation='relu'))
        else:
          model.add(GRU(unit, dropout = drop, recurrent_dropout= drop,
                        input_shape=(train_feature.shape[1], train_feature.shape[2]), 
                        return_sequences = True,
                        activation='relu'))
          model.add(GRU(unit, dropout = drop, recurrent_dropout=drop, return_sequences=False, stateful=False))

        model.add(Dense(1))
      
        
        model.compile(loss='mean_squared_error', optimizer='adam')

        early_stop = EarlyStopping(monitor='val_loss', patience=10)

        import os
        from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

        model_path = 'model'
        filename = os.path.join(model_path, 'tmp_checkpoint.h5')
        checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

        history = model.fit(x_train, y_train, 
                                            epochs=100, 
                                            batch_size=16,
                                            validation_data=(x_valid, y_valid), 
                                            callbacks=[early_stop, checkpoint])


        pred = model.predict(test_feature)
        # rescaleing 작업
        rescaled_y_true = scaler1.inverse_transform(np.array(test_label).reshape(-1, 1))
        rescaled_pred = scaler1.inverse_transform(np.array(pred).reshape(-1,1))
        
        
        # 평가지표(RMSE) 계산
        RMSE = np.sqrt(mean_squared_error(rescaled_y_true, rescaled_pred))
        result.append(RMSE)
        print(f"{i * 1000}길이의 데이터 적용 완료\n 길이: {i * 1000}, RMSE:{RMSE}")
        print(result)
        
        
        model.load_weights(filename)

        new_pred = model.predict(pred_feature)
        rescaled_newpred = scaler1.inverse_transform(np.array(new_pred).reshape(-1,1))
        predict.append(rescaled_newpred)
      df_result.loc[idx] = [unit, drop, layer, 1000*result.index(min(result)), min(result), predict[result.index(min(result))]]
      idx += 1
      df_result



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28985 sha256=4b14549ef5003df85de4c5894e2d4e47ac018daf4069758cb20c66b78a04daaf
  Stored in directory: /root/.cache/pip/wheels/bc/2a/c2/a56e77d07edc16a1fa7fb012667e55cb0643cfa65996bddecc
Successfully built ta


/usr/local/lib/python3.7/dist-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.7/dist-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Epoch 1/100
27/34 [======================>.......] - ETA: 0s - loss: 0.0110
Epoch 1: val_loss improved from inf to 0.00629, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 2s 16ms/step - loss: 0.0094 - val_loss: 0.0063
Epoch 2/100
29/34 [========================>.....] - ETA: 0s - loss: 0.0029
Epoch 2: val_loss did not improve from 0.00629
34/34 [==============================] - 0s 7ms/step - loss: 0.0028 - val_loss: 0.0092
Epoch 3/100
34/34 [==============================] - ETA: 0s - loss: 0.0018
Epoch 3: val_loss did not improve from 0.00629
34/34 [==============================] - 0s 8ms/step - loss: 0.0018 - val_loss: 0.0066
Epoch 4/100
31/34 [==========================>...] - ETA: 0s - loss: 0.0014
Epoch 4: val_loss did not improve from 0.00629
34/34 [==============================] - 0s 8ms/step - loss: 0.0014 - val_loss: 0.0064
Epoch 5/100
27/34 [======================>.......] - ETA: 0s - loss: 0.0013
Epoch 5: val_loss improved from 0.00629 to

/usr/local/lib/python3.7/dist-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.7/dist-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Epoch 1/100
31/34 [==========================>...] - ETA: 0s - loss: 0.0211
Epoch 1: val_loss improved from inf to 0.00290, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 4s 33ms/step - loss: 0.0195 - val_loss: 0.0029
Epoch 2/100
31/34 [==========================>...] - ETA: 0s - loss: 0.0024
Epoch 2: val_loss did not improve from 0.00290
34/34 [==============================] - 0s 13ms/step - loss: 0.0024 - val_loss: 0.0043
Epoch 3/100
30/34 [=========================>....] - ETA: 0s - loss: 0.0014
Epoch 3: val_loss improved from 0.00290 to 0.00259, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 0s 13ms/step - loss: 0.0014 - val_loss: 0.0026
Epoch 4/100
34/34 [==============================] - ETA: 0s - loss: 0.0013
Epoch 4: val_loss did not improve from 0.00259
34/34 [==============================] - 0s 13ms/step - loss: 0.0013 - val_loss: 0.0040
Epoch 5/100
34/34 [==============================] - ETA: 0s - loss: 9

...

Epoch 1/100
34/34 [==============================] - ETA: 0s - loss: 0.0229
Epoch 1: val_loss improved from inf to 0.01691, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 6s 59ms/step - loss: 0.0229 - val_loss: 0.0169
Epoch 2/100
34/34 [==============================] - ETA: 0s - loss: 0.0047
Epoch 2: val_loss did not improve from 0.01691
34/34 [==============================] - 1s 40ms/step - loss: 0.0047 - val_loss: 0.0242
Epoch 3/100
34/34 [==============================] - ETA: 0s - loss: 0.0039
Epoch 3: val_loss improved from 0.01691 to 0.01471, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 1s 42ms/step - loss: 0.0039 - val_loss: 0.0147
Epoch 4/100
34/34 [==============================] - ETA: 0s - loss: 0.0032
Epoch 4: val_loss improved from 0.01471 to 0.01160, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 1s 42ms/step - loss: 0.0032 - val_loss: 0.0116
Epoch 5/100
34/34 [======

/usr/local/lib/python3.7/dist-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.7/dist-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Epoch 1/100
33/34 [============================>.] - ETA: 0s - loss: 0.0353
Epoch 1: val_loss improved from inf to 0.01166, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 3s 48ms/step - loss: 0.0344 - val_loss: 0.0117
Epoch 2/100
34/34 [==============================] - ETA: 0s - loss: 0.0057
Epoch 2: val_loss improved from 0.01166 to 0.00468, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 1s 37ms/step - loss: 0.0057 - val_loss: 0.0047
Epoch 3/100
33/34 [============================>.] - ETA: 0s - loss: 0.0038
Epoch 3: val_loss improved from 0.00468 to 0.00240, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 1s 36ms/step - loss: 0.0038 - val_loss: 0.0024
Epoch 4/100
33/34 [============================>.] - ETA: 0s - loss: 0.0031
Epoch 4: val_loss improved from 0.00240 to 0.00166, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 1s 37ms/step - loss: 0.0031

/usr/local/lib/python3.7/dist-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.7/dist-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


Epoch 1/100
34/34 [==============================] - ETA: 0s - loss: 0.0227
Epoch 1: val_loss improved from inf to 0.03506, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 7s 109ms/step - loss: 0.0227 - val_loss: 0.0351
Epoch 2/100
34/34 [==============================] - ETA: 0s - loss: 0.0051
Epoch 2: val_loss improved from 0.03506 to 0.00851, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 3s 81ms/step - loss: 0.0051 - val_loss: 0.0085
Epoch 3/100
34/34 [==============================] - ETA: 0s - loss: 0.0029
Epoch 3: val_loss improved from 0.00851 to 0.00190, saving model to model/tmp_checkpoint.h5
34/34 [==============================] - 3s 83ms/step - loss: 0.0029 - val_loss: 0.0019
Epoch 4/100
34/34 [==============================] - ETA: 0s - loss: 0.0028
Epoch 4: val_loss did not improve from 0.00190
34/34 [==============================] - 3s 82ms/step - loss: 0.0028 - val_loss: 0.0022
Epoch 5/100
34/34 [=====

In [4]:
df_result = df_result.sort_values(['유닛수', '층 개수', 'DropOut'])
df_result = df_result.reset_index(drop = True)
df_result

,유닛수,DropOut,층 개수,최적 개수,최소 rmse,최적 예측값
0,16,0,1,0,38125.867220,[[782476.8]]
1,16,0.2,1,0,54022.338697,[[805630.5]]
2,16,0.3,1,0,41788.504436,[[873507.75]]
3,16,0,2,0,36400.378599,[[804213.44]]
4,16,0.2,2,0,48437.196218,[[811999.56]]
5,16,0.3,2,0,38546.724833,[[814735.5]]
6,64,0,1,1000,26135.691903,[[845442.94]]
7,64,0.2,1,0,31042.993465,[[867202.8]]
8,64,0.3,1,0,37543.034858,[[848847.06]]
9,64,0,2,0,48558.501934,[[798277.9]]


In [5]:
df_result.to_csv("gru_삼성바이오로직스_recurDropout.csv")